```{contents}
```
## Multi-Step Pipeline 

A **multi-step pipeline** in LangGraph is a structured sequence of processing stages—each stage implemented as a **node**—that transforms an input through multiple reasoning, tool, and validation phases under **explicit state control**.
It generalizes traditional pipelines by allowing **branching, looping, human intervention, persistence, and recovery**.

---

### **1. Why Multi-Step Pipelines Matter**

Real-world AI tasks are rarely single-shot:

| Requirement      | Example                    |
| ---------------- | -------------------------- |
| Data preparation | Clean → validate → enrich  |
| Reasoning        | Plan → execute → verify    |
| Tool usage       | Search → compute → analyze |
| Quality control  | Draft → critique → revise  |
| Governance       | Approve → audit → publish  |

LangGraph models these workflows as **explicit computational graphs** rather than fragile linear scripts.

---

### **2. Core Pipeline Abstractions**

| Concept        | LangGraph Mapping  |
| -------------- | ------------------ |
| Stage          | Node               |
| Pipeline flow  | Directed edges     |
| Shared context | State              |
| Routing logic  | Conditional edges  |
| Iteration      | Cycles             |
| Validation     | Gate nodes         |
| Persistence    | Checkpointed state |

---

### **3. Conceptual Structure**

```
Input
  ↓
Preprocess
  ↓
Plan
  ↓
Execute
  ↓
Validate ──┐
  ↓        │
Publish ←──┘ (optional refinement loop)
```

---

### **4. Defining Pipeline State**

```python
from typing import TypedDict, List

class State(TypedDict):
    query: str
    plan: str
    result: str
    verified: bool
```

State is **the contract** between stages.

---

### **5. Implementing a Multi-Step Pipeline**

```python
from langgraph.graph import StateGraph, END

def preprocess(state):
    return {"query": state["query"].strip()}

def plan(state):
    return {"plan": f"Steps for: {state['query']}"}

def execute(state):
    return {"result": f"Executed {state['plan']}"}

def validate(state):
    ok = "Executed" in state["result"]
    return {"verified": ok}

def route(state):
    return END if state["verified"] else "plan"
```

### **Pipeline Construction**

```python
builder = StateGraph(State)

builder.add_node("preprocess", preprocess)
builder.add_node("plan", plan)
builder.add_node("execute", execute)
builder.add_node("validate", validate)

builder.set_entry_point("preprocess")
builder.add_edge("preprocess", "plan")
builder.add_edge("plan", "execute")
builder.add_edge("execute", "validate")

builder.add_conditional_edges("validate", route, {
    "plan": "plan",
    END: END
})

graph = builder.compile()
```

### **Execution**

```python
result = graph.invoke({"query": "Build a pricing model"})
print(result)
```

---

### **6. Pipeline Variants**

| Variant                | Purpose                      |
| ---------------------- | ---------------------------- |
| Linear pipeline        | Deterministic ETL-style flow |
| Branching pipeline     | Dynamic routing              |
| Cyclic pipeline        | Iterative refinement         |
| Parallel pipeline      | Fan-out/fan-in processing    |
| Human-in-loop pipeline | Approval & correction        |
| Autonomous pipeline    | Self-directed execution      |

---

### **7. Production Enhancements**

| Feature            | Benefit         |
| ------------------ | --------------- |
| Checkpointing      | Crash recovery  |
| Retries & timeouts | Fault tolerance |
| Human gates        | Governance      |
| Tracing            | Observability   |
| Cost control       | Budget safety   |
| Versioned graphs   | Safe deployment |

---

### **8. Mental Model**

A LangGraph multi-step pipeline behaves like a **distributed workflow engine**:

> **State + Graph + Control Logic = Robust AI System**

This design enables **scalable, auditable, self-correcting AI pipelines** suitable for enterprise workloads.


### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# -------------------------
# 1. Define Pipeline State
# -------------------------
class State(TypedDict):
    query: str
    plan: str
    result: str
    verified: bool

# -------------------------
# 2. Define Pipeline Stages
# -------------------------
def preprocess(state: State):
    return {"query": state["query"].strip()}

def plan(state: State):
    return {"plan": f"Steps for: {state['query']}"}

def execute(state: State):
    return {"result": f"Executed -> {state['plan']}"}

def validate(state: State):
    ok = "Executed" in state["result"]
    return {"verified": ok}

def router(state: State):
    return END if state["verified"] else "plan"

# -------------------------
# 3. Build Graph
# -------------------------
builder = StateGraph(State)

builder.add_node("preprocess", preprocess)
builder.add_node("plan", plan)
builder.add_node("execute", execute)
builder.add_node("validate", validate)

builder.set_entry_point("preprocess")
builder.add_edge("preprocess", "plan")
builder.add_edge("plan", "execute")
builder.add_edge("execute", "validate")

builder.add_conditional_edges("validate", router, {
    "plan": "plan",
    END: END
})

graph = builder.compile()

# -------------------------
# 4. Run Pipeline
# -------------------------
output = graph.invoke({"query": "Design an ML system"})
print(output)


{'query': 'Design an ML system', 'plan': 'Steps for: Design an ML system', 'result': 'Executed -> Steps for: Design an ML system', 'verified': True}
